In [2]:

!pip install flask-ngrok
!pip install pandas numpy scikit-learn-


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


ERROR: Invalid requirement: 'scikit-learn-': Expected end or semicolon (after name and no valid version specifier)
    scikit-learn-
                ^


In [4]:
pip install flask_ngrok


Note: you may need to restart the kernel to use updated packages.


In [6]:
pip show flask_ngrok


Name: flask-ngrok
Version: 0.0.25
Summary: A simple way to demo Flask apps from your machine.
Home-page: https://github.com/gstaff/flask-ngrok
Author: Grant Stafford
Author-email: 
License: UNKNOWN
Location: C:\Users\DELL\anaconda3\Lib\site-packages
Requires: Flask, requests
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from flask import Flask, render_template_string, request
from flask_ngrok import run_with_ngrok  # To run the app via ngrok


In [3]:
# Load the UCI Heart Disease dataset from local path
url = r"D:\svm\heart+disease (1)\processed.cleveland.data"

# Columns as per UCI dataset description
columns = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 
           'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']

# Load the dataset, replacing '?' with NaN (missing values)
data = pd.read_csv(url, names=columns)
data = data.replace('?', np.nan)

# Drop rows with missing values
data.dropna(inplace=True)

# Convert 'target' to binary: 0 = no disease, 1+ = disease
data['target'] = data['target'].apply(lambda x: 1 if x > 0 else 0)

# Select 4 important features and target
X = data[['age', 'cp', 'thalach', 'oldpeak']]  # Selected important features
y = data['target']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train an SVM model
model = SVC(kernel='linear', probability=True)
model.fit(X_train, y_train)

# Save the model to a file for future use
with open('heart_disease_svm_model.pkl', 'wb') as file:
    pickle.dump(model, file)


In [ ]:
# Flask web application
app = Flask(__name__)
run_with_ngrok(app)  # Use ngrok to run the app

# Load the trained model
with open('heart_disease_svm_model.pkl', 'rb') as file:
    model = pickle.load(file)

# Home route for input form
@app.route('/')
def home():
    return '''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Heart Disease Prediction</title>
        <style>
            body { font-family: Arial, sans-serif; margin: 50px; }
            .container { max-width: 600px; margin: 0 auto; }
            h1 { text-align: center; }
            form { display: flex; flex-direction: column; }
            label, input { margin: 10px 0; }
            input[type="submit"] { background-color: #4CAF50; color: white; padding: 10px; border: none; cursor: pointer; }
            input[type="submit"]:hover { background-color: #45a049; }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Heart Disease Prediction</h1>
            <form action="/predict" method="POST">
                <label for="age">Age:</label>
                <input type="number" id="age" name="age" required>

                <label for="cp">Chest Pain Type (0-3):</label>
                <input type="number" id="cp" name="cp" required min="0" max="3">

                <label for="thalach">Max Heart Rate:</label>
                <input type="number" id="thalach" name="thalach" required>

                <label for="oldpeak">Oldpeak (ST depression):</label>
                <input type="number" id="oldpeak" name="oldpeak" required step="0.1">

                <input type="submit" value="Predict">
            </form>
        </div>
    </body>
    </html>
    '''

# Prediction route to display result
@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get form data for 4 features
        features = [
            float(request.form['age']),
            float(request.form['cp']),
            float(request.form['thalach']),
            float(request.form['oldpeak']),
        ]
        
        # Convert to numpy array and reshape
        features = np.array(features).reshape(1, -1)

        # Scale the features like during training
        features = scaler.transform(features)

        # Make the prediction
        prediction = model.predict(features)
        result = "Heart Disease Detected" if prediction[0] == 1 else "No Heart Disease"
    except Exception as e:
        result = f"Error: {e}"

    # Return the result in HTML
    return f'''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Prediction Result</title>
    </head>
    <body>
        <h1>Prediction Result</h1>
        <p>{result}</p>
        <a href="/">Go Back</a>
    </body>
    </html>
    '''

if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
Exception in thread Thread-5:
Traceback (most recent call last):
  File "C:\Users\DELL\anaconda3\Lib\site-packages\urllib3\connection.py", line 196, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\DELL\anaconda3\Lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    raise err
  File "C:\Users\DELL\anaconda3\Lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\DELL\anaconda3\Lib\site-packages\urllib3\connectionpool.py", line 789, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\DELL\anaconda3\Lib\site-packages\urlli